![](logo1.jpg)

# **shAI Training 2023 | Level 1**

## Task #8 (End-to-End ML Project {part_2})

## Welcome to the exercises for reviewing second part of end to end ML project.
**Make sure that you read and understand ch2 from the hands-on ML book (page 72 to the end of the chapter ) before start with this notebook.**

**If you stuck with anything reread that part from the book and feel free to ask about anything in the messenger group as you go along.**

 ## Good Luck : )

## first run the following cell for the first part of the project to continue your work

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
import os
import tarfile
import urllib
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

def load_housing_data(housing_path=HOUSING_PATH):
   csv_path = os.path.join(housing_path, "housing.csv")
   return pd.read_csv(csv_path)

fetch_housing_data()
housing = load_housing_data()

rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs)])

housing_prepared = full_pipeline.fit_transform(housing)

# 1- Select and Train a Model

# Let’s first train a LinearRegression model

In [3]:
# CODE HERE
from sklearn.linear_model import LinearRegression

# Initialize and train the Linear Regression model
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)


LinearRegression()

# First try it out on a few instances from the training set:


In [4]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5].values
some_data_prepared = full_pipeline.transform(some_data)
housing_predictions = lin_reg.predict(some_data_prepared).round(-2)
print("Predictions:", housing_predictions)
print("Labels:", list(some_labels))

Predictions: [181700. 290600. 245000. 146500. 163200.]
Labels: [103000.0, 382100.0, 172600.0, 93400.0, 96500.0]


# measure this regression model’s RMSE on the whole training set
* sing Scikit-Learn’s mean_squared_error() function:

In [5]:
from sklearn.metrics import mean_squared_error

In [9]:
# CODE HERE
from sklearn.metrics import mean_squared_error
import numpy as np
housing_predictions = lin_reg.predict(housing_prepared)
mse = mean_squared_error(housing_labels, housing_predictions)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE) on the whole training set:", rmse)

Root Mean Squared Error (RMSE) on the whole training set: 67593.20745775253


# judge on the RMSE result for this model
not a great score: most districts
median_housing_values range between $120,000  and  $265,000, so a
typical prediction error of $67,593 is really not very satisfying

# Let’s train a Decision Tree Regressor model
## more powerful model

In [10]:
from sklearn.tree import DecisionTreeRegressor

In [11]:
# CODE HERE
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)



DecisionTreeRegressor(random_state=42)

# Now evaluate the model on the training set
* using Scikit-Learn’s mean_squared_error() function:

In [12]:
# CODE HERE
housing_predictions_tree = tree_reg.predict(housing_prepared)
mse_tree = mean_squared_error(housing_labels, housing_predictions_tree)
rmse_tree = np.sqrt(mse_tree)
print("Root Mean Squared Error (RMSE) on the whole training set (Decision Tree):", rmse_tree)

Root Mean Squared Error (RMSE) on the whole training set (Decision Tree): 0.0


# Explaine this result
write down your answar


model has badly overfit the data

# Evaluation Using Cross-Validation

1-split the training set into 10 distinct subsets then train and evaluate the Decision Tree model

In [13]:
from sklearn.model_selection import cross_val_score

In [14]:
# CODE HERE
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)

# Calculate the root mean squared error from the cross-validation scores
rmse_scores = np.sqrt(-scores)

2- display the resultant scores and calculate its Mean and Standard deviation

In [15]:
# CODE HERE
print(pd.Series(-scores).describe())


count    1.000000e+01
mean     4.746728e+09
std      3.775429e+08
min      4.265770e+09
25%      4.505264e+09
50%      4.697165e+09
75%      4.952104e+09
max      5.481225e+09
dtype: float64


3-repaet the same steps to compute the same scores for the Linear Regression  model

*notice the difference between the results of the two models*

In [16]:
# CODE HERE
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
print(pd.Series(-lin_scores).describe())


count    1.000000e+01
mean     4.606782e+09
std      3.567518e+08
min      4.225088e+09
25%      4.286732e+09
50%      4.592179e+09
75%      4.740271e+09
max      5.291089e+09
dtype: float64


## Let’s train one last model the RandomForestRegressor.

In [17]:
# CODE HERE
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor(random_state=42)

# repeat the same steps to compute the same scores its Mean and Standard deviation for the Random Forest model

In [18]:
# CODE HERE
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
print(pd.Series(-forest_scores).describe())


count    1.000000e+01
mean     2.472310e+09
std      2.021539e+08
min      2.208057e+09
25%      2.280015e+09
50%      2.477416e+09
75%      2.639911e+09
max      2.784875e+09
dtype: float64


# Save every model you experiment with
*using the joblib library*

In [19]:
# CODE HERE
from joblib import dump
dump(lin_reg, 'linear_regression_model.joblib')
dump(tree_reg, 'decision_tree_regressor_model.joblib')
dump(forest_reg, 'random_forest_regressor_model.joblib')

['random_forest_regressor_model.joblib']

## now you have a shortlist of promising models. You now need to
## fine-tune them!
# Fine-Tune Your Model

## 1- Grid Search
## evaluate all the possible combinations of hyperparameter values for the RandomForestRegressor
*It may take a long time*

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
# Define the grid of hyperparameters
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

# Initialize the RandomForestRegressor model
forest_reg = RandomForestRegressor(random_state=42)

# Create the grid search object
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# Perform the grid search to find the best hyperparameters
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

with the evaluation scores

In [22]:
# CODE HERE
print("Best hyperparameters:", grid_search.best_params_)
print("Best estimator:", grid_search.best_estimator_)

Best hyperparameters: {'max_features': 8, 'n_estimators': 30}
Best estimator: RandomForestRegressor(max_features=8, n_estimators=30, random_state=42)


# Analyze the Best Models and Their Errors
1-indicate the relative importance of each attribute

In [23]:
# CODE HERE
best_forest_reg = grid_search.best_estimator_
best_forest_reg.fit(housing_prepared, housing_labels)

# Get feature importances
feature_importances = best_forest_reg.feature_importances_

# Get the list of attribute names
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs

2-display these importance scores next to their corresponding attribute names:

In [24]:
# CODE HERE
sorted(zip(feature_importances, attributes), reverse=True)


[(0.37183388814667373, 'median_income'),
 (0.16501259905352061, 'INLAND'),
 (0.1097583570805625, 'pop_per_hhold'),
 (0.06844933917056337, 'longitude'),
 (0.06491313404989336, 'latitude'),
 (0.061176949819538154, 'bedrooms_per_room'),
 (0.04945029095965893, 'rooms_per_hhold'),
 (0.0417428332867604, 'housing_median_age'),
 (0.014515821649955954, 'total_rooms'),
 (0.014300165080528202, 'population'),
 (0.013706064997348836, 'total_bedrooms'),
 (0.012959133102106802, 'households'),
 (0.0073955403637295455, '<1H OCEAN'),
 (0.0027172702007613975, 'NEAR OCEAN'),
 (0.0018399449484004105, 'NEAR BAY'),
 (0.00022866808999787155, 'ISLAND')]

## Now is the time to evaluate the final model on the test set.
# Evaluate Your System on the Test Set

1-get the predictors and the labels from your test set

In [25]:
# CODE HERE
X_test = test_set.drop("median_house_value", axis=1)
y_test = test_set["median_house_value"].copy()

2-run your full_pipeline to transform the data

In [26]:
# CODE HERE
X_test_prepared = full_pipeline.transform(X_test)


3-evaluate the final model on the test set

In [28]:
# CODE HERE
final_predictions = best_forest_reg.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)

final_rmse = np.sqrt(final_mse)
print("Final RMSE on the test set:", final_rmse)

Final RMSE on the test set: 49198.020631676336


# compute a 95% confidence interval for the generalization error
*using scipy.stats.t.interval():*

In [29]:
from scipy import stats

In [30]:
# CODE HERE
num_samples = len(y_test)

degrees_of_freedom = num_samples - 1
standard_error = np.sqrt(final_mse / num_samples)
confidence_interval = stats.t.interval(0.95, degrees_of_freedom, loc=final_rmse, scale=standard_error)

print("95% Confidence Interval for Generalization Error:", confidence_interval)

95% Confidence Interval for Generalization Error: (47696.76978712008, 50699.27147623259)


# Great Job!
# #shAI_Club